In [1]:
print("OK!")

OK!


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers





ModuleNotFoundError: No module named 'pinecone'

In [ ]:
pc_clef='pcsk_33dJwb_LuHpCRjZCZsktMRxbfJuVub362oFXVMLn8ddMGnXEpoXzL815Q9Vx72HBBAHr8M'

